# DREAM in WFM mode

In this notebook, we show how to use Scippneutron's `tof` module to find the boundaries of the WFM frames for the DREAM instrument.
We then apply a time correction to each frame to compute a time-of-flight coordinate, from which a wavelength can be derived.

In [ ]:
import plopp as pp
import scipp as sc
import sciline as sl
from scippneutron.chopper import DiskChopper
from scippneutron.tof import unwrap

## Setting up the beamline

### Creating the beamline choppers

We begin by defining the chopper settings for our beamline.
In principle, the chopper setting could simply be read from a NeXus file.

The DREAM instrument has

- 2 pulse-shaping choppers (PSC)
- 1 overlap chopper (OC)
- 1 band-control chopper (BCC)
- 1 T0 chopper

In [ ]:
psc1 = DiskChopper(
    frequency=sc.scalar(14.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(286 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 6.145], unit="m"),
    slit_begin=sc.array(
        dims=["cutout"],
        values=[-1.23, 70.49, 84.765, 113.565, 170.29, 271.635, 286.035, 301.17],
        unit="deg",
    ),
    slit_end=sc.array(
        dims=["cutout"],
        values=[1.23, 73.51, 88.035, 116.835, 175.31, 275.565, 289.965, 303.63],
        unit="deg",
    ),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

psc2 = DiskChopper(
    frequency=sc.scalar(-14.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(-236, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 6.155], unit="m"),
    slit_begin=sc.array(
        dims=["cutout"],
        values=[-1.23, 27.0, 55.8, 142.385, 156.765, 214.115, 257.23, 315.49],
        unit="deg",
    ),
    slit_end=sc.array(
        dims=["cutout"],
        values=[1.23, 30.6, 59.4, 145.615, 160.035, 217.885, 261.17, 318.11],
        unit="deg",
    ),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

oc = DiskChopper(
    frequency=sc.scalar(14.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(297 - 180 - 90, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 6.174], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-27.6 * 0.5], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[27.6 * 0.5], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

bcc = DiskChopper(
    frequency=sc.scalar(112.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(240 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 9.78], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-36.875, 143.125], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[36.875, 216.875], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

t0 = DiskChopper(
    frequency=sc.scalar(28.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(280 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 13.05], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-314.9 * 0.5], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[314.9 * 0.5], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

disk_choppers = {"psc1": psc1, "psc2": psc2, "oc": oc, "bcc": bcc, "t0": t0}

It is possible to visualize the properties of the choppers by inspecting their `repr`:

In [ ]:
psc2

### Adding a detector

In [ ]:
Ltotal = sc.scalar(76.55 + 1.125, unit="m")

## Creating some neutron events

We create a semi-realistic set of neutron events based on the ESS pulse.

In [ ]:
from scippneutron.tof.fakes import FakeBeamlineEss

ess_beamline = FakeBeamlineEss(
    choppers=disk_choppers,
    monitors={"detector": Ltotal},
    run_length=sc.scalar(1 / 14, unit="s") * 4,
    events_per_pulse=200_000,
)

The initial birth times and wavelengths of the generated neutrons can be visualized (for a single pulse):

In [ ]:
one_pulse = ess_beamline.source.data["pulse", 0]
one_pulse.hist(time=300).plot() + one_pulse.hist(wavelength=300).plot()

In [ ]:
ess_beamline.model_result.plot()

From this fake beamline, we extract the raw neutron signal at our detector:

In [ ]:
raw_data = ess_beamline.get_monitor("detector")[0]

# Visualize
raw_data.hist(event_time_offset=300).sum("pulse").plot()

The total number of neutrons in our sample data that make it through the to detector is:

In [ ]:
raw_data.sum().value

## Computing time-of-flight

The chopper information is next used to construct a lookup table that provides an estimate of the real time-of-flight as a function of neutron time-of-arrival.

We use the `tof` module to propagate a pulse of neutrons through the chopper system to the detectors,
and predict the most likely neutron wavelength for a given time-of-arrival and distance from source.

From this,
we build a lookup table on which bilinear interpolation is used to compute a wavelength (and its corresponding time-of-flight)
for every neutron event.

### Setting up the workflow

In [ ]:
workflow = sl.Pipeline(unwrap.providers(), params=unwrap.params())

workflow[unwrap.Facility] = "ess"
workflow[unwrap.Choppers] = disk_choppers
workflow[unwrap.RawData] = raw_data
workflow[unwrap.Ltotal] = Ltotal

workflow.visualize(unwrap.TofData)

### Inspecting the lookup table

The workflow first runs a `tof` simulation using the chopper parameters above,
and the result is stored in `SimulationResults` (see graph above).

From these simulated neutrons, we create figures displaying the neutron wavelengths and time-of-flight,
as a function of arrival time at the detector.

This is the basis for creating our lookup table.

In [ ]:
sim = workflow.compute(unwrap.SimulationResults)
# Compute time-of-arrival at the detector
tarrival = sim.time_of_arrival + ((Ltotal - sim.distance) / sim.speed).to(unit="us")
# Compute time-of-flight at the detector
tflight = (Ltotal / sim.speed).to(unit="us")

events = sc.DataArray(
    data=sim.weight,
    coords={"wavelength": sim.wavelength, "toa": tarrival, "tof": tflight},
)
fig1 = events.hist(wavelength=300, toa=300).plot(norm="log")
fig2 = events.hist(tof=300, toa=300).plot(norm="log")
fig1 + fig2

The lookup table is then obtained by computing the weighted mean of the time-of-flight inside each time-of-arrival bin.

This is illustrated by the orange line in the figure below:

In [ ]:
table = workflow.compute(unwrap.TimeOfFlightLookupTable)

# Overlay mean on the figure above
table["distance", 1].plot(ax=fig2.ax, color="C1")

### Computing a time-of-flight coordinate

We will now use our workflow to obtain our event data with a time-of-flight coordinate:

In [ ]:
tofs = workflow.compute(unwrap.TofData)
tofs

Histogramming the data for a plot should show a profile with 6 bumps that correspond to the frames:

In [ ]:
tofs.bins.concat().hist(tof=300).plot()

### Converting to wavelength

We can now convert our new time-of-flight coordinate to a neutron wavelength, using `tranform_coords`:

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

# Perform coordinate transformation
graph = {**beamline(scatter=False), **elastic("tof")}
wav_wfm = tofs.transform_coords("wavelength", graph=graph)

# Define wavelength bin edges
wavs = sc.linspace("wavelength", 0.8, 4.6, 201, unit="angstrom")

wav_wfm.hist(wavelength=wavs).sum("pulse").plot()

### Comparing to the ground truth

As a consistency check, because we actually know the wavelengths of the neutrons we created,
we can compare the true neutron wavelengths to those we computed above.

In [ ]:
ground_truth = ess_beamline.model_result["detector"].data.flatten(to="event")
ground_truth = ground_truth[~ground_truth.masks["blocked_by_others"]]

pp.plot(
    {
        "wfm": wav_wfm.hist(wavelength=wavs).sum("pulse"),
        "ground_truth": ground_truth.hist(wavelength=wavs),
    }
)

## Multiple detector pixels

It is also possible to compute the neutron time-of-flight for multiple detector pixels at once,
where every pixel has different frame bounds
(because every pixel is at a different distance from the source).

In our setup, we simply propagate the same neutrons to multiple detector pixels,
as if they were not absorbed by the first pixel they meet.

In [ ]:
Ltotal = sc.array(dims=["detector_number"], values=[77.675, 76.0], unit="m")
monitors = {f"detector{i}": ltot for i, ltot in enumerate(Ltotal)}

ess_beamline = FakeBeamlineEss(
    choppers=disk_choppers,
    monitors=monitors,
    run_length=sc.scalar(1 / 14, unit="s") * 4,
    events_per_pulse=200_000,
)

Our raw data has now a `detector_number` dimension of length 2.

We can plot the neutron `event_time_offset` for the two detector pixels and see that the offsets are shifted to the left for the pixel that is closest to the source.

In [ ]:
raw_data = sc.concat(
    [ess_beamline.get_monitor(key)[0] for key in monitors.keys()],
    dim="detector_number",
)

# Visualize
pp.plot(
    sc.collapse(
        raw_data.hist(event_time_offset=300).sum("pulse"), keep="event_time_offset"
    )
)

Computing time-of-flight is done in the same way as above.
We need to remember to update our workflow:

In [ ]:
# Update workflow
workflow[unwrap.Ltotal] = Ltotal
workflow[unwrap.RawData] = raw_data

# Compute tofs and wavelengths
tofs = workflow.compute(unwrap.TofData)
wav_wfm = tofs.transform_coords("wavelength", graph=graph)

# Compare in plot
ground_truth = []
for det in ess_beamline.monitors:
    data = ess_beamline.model_result[det.name].data.flatten(to="event")
    ground_truth.append(data[~data.masks["blocked_by_others"]])

figs = [
    pp.plot(
        {
            "wfm": wav_wfm["detector_number", i].bins.concat().hist(wavelength=wavs),
            "ground_truth": ground_truth[i].hist(wavelength=wavs),
        },
        title=f"Pixel {i+1}",
    )
    for i in range(len(Ltotal))
]

figs[0] + figs[1]

## Handling time overlap between subframes

In some (relatively rare) cases, where a chopper cascade is slightly ill-defined,
it is sometimes possible for some subframes to overlap in time with other subframes.

This is basically when neutrons passed through different pulse-shaping chopper openings,
but arrive at the same time at the detector.

In this case, it is actually not possible to accurately determine the wavelength of the neutrons.
ScippNeutron handles this by masking the overlapping regions and throwing away any neutrons that lie within it.

To simulate this, we modify slightly the phase and the cutouts of the band-control chopper:

In [ ]:
disk_choppers["bcc"] = DiskChopper(
    frequency=sc.scalar(112.0, unit="Hz"),
    beam_position=sc.scalar(0.0, unit="deg"),
    phase=sc.scalar(240 - 180, unit="deg"),
    axle_position=sc.vector(value=[0, 0, 9.78], unit="m"),
    slit_begin=sc.array(dims=["cutout"], values=[-36.875, 143.125], unit="deg"),
    slit_end=sc.array(dims=["cutout"], values=[46.875, 216.875], unit="deg"),
    slit_height=sc.scalar(10.0, unit="cm"),
    radius=sc.scalar(30.0, unit="cm"),
)

# Go back to a single detector pixel
Ltotal = sc.scalar(76.55 + 1.125, unit="m")

ess_beamline = FakeBeamlineEss(
    choppers=disk_choppers,
    monitors={"detector": Ltotal},
    run_length=sc.scalar(1 / 14, unit="s") * 4,
    events_per_pulse=200_000,
)

# Update workflow
workflow[unwrap.Ltotal] = Ltotal
workflow[unwrap.RawData] = ess_beamline.get_monitor("detector")[0]

ess_beamline.model_result.plot()

We can now see that there is no longer a gap between the two frames at the center of each pulse (green region).

Another way of looking at this is looking at the wavelength vs time-of-arrival plot,
which also shows overlap in time at the junction between the two frames:

In [ ]:
sim = workflow.compute(unwrap.SimulationResults)
# Compute time-of-arrival at the detector
tarrival = sim.time_of_arrival + ((Ltotal - sim.distance) / sim.speed).to(unit="us")
# Compute time-of-flight at the detector
tflight = (Ltotal / sim.speed).to(unit="us")

events = sc.DataArray(
    data=sim.weight,
    coords={"wavelength": sim.wavelength, "toa": tarrival, "tof": tflight},
)
events.hist(wavelength=300, toa=300).plot(norm="log")

The data in the lookup table contains both the mean time-of-flight for each distance and time-of-arrival bin,
but also the variance inside each bin.

In the regions where there is no time overlap,
the variance is small (the regions are close to a thin line).
However, in the central region where overlap occurs,
we are computing a mean between two regions which have similar 'brightness'.

This leads to a large variance, and this is visible when plotting the variances on a 2D figure.

In [ ]:
table = workflow.compute(unwrap.TimeOfFlightLookupTable)
table.plot() / sc.variances(table).plot(norm="log")

The workflow has a parameter which is used to mask out regions where the variance is above a certain threshold.

It is difficult to automatically detector this threshold,
as it can vary a lot depending on how much signal is received by the detectors,
and how far the detectors are from the source.
It is thus more robust to simply have a user tunable parameter on the workflow.

In [ ]:
workflow[unwrap.LookupTableVarianceThreshold] = 1.0e-3

workflow.compute(unwrap.MaskedTimeOfFlightLookupTable).plot()

We can now see that the central region is masked out.

The neutrons in that region will be discarded in the time-of-flight calculation
(in practice, they are given a NaN value as a time-of-flight).

This is visible when comparing to the true neutron wavelengths,
where we see that some counts were lost between the two frames.

In [ ]:
# Compute time-of-flight
tofs = workflow.compute(unwrap.TofData)
# Compute wavelength
wav_wfm = tofs.transform_coords("wavelength", graph=graph)

# Compare to the true wavelengths
ground_truth = ess_beamline.model_result["detector"].data.flatten(to="event")
ground_truth = ground_truth[~ground_truth.masks["blocked_by_others"]]

pp.plot(
    {
        "wfm": wav_wfm.hist(wavelength=wavs).sum("pulse"),
        "ground_truth": ground_truth.hist(wavelength=wavs),
    }
)